In [1]:
!pip install torch

    100% |████████████████████████████████| 591.8MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6118c000 @  0x7fef41f4a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
!ls /gdrive/My\ Drive/data

 _about.txt			 fra-eng.zip	     tarantino.csv
'aesw2016(v1.2)_train.xml'	 fra.txt
'aesw2016(v1.2)_train.xml.bz2'	 glove.6B.300d.txt


In [4]:
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
!python -m spacy download fr

    100% |████████████████████████████████| 39.8MB 45.7MB/s 
  Running setup.py install for fr-core-news-sm ... - \ | / done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/fr

    You can now load the model via spacy.load('fr')



In [67]:
s = set()
l1 = ["<start>", "courez", "<end>"]
l2 = ["<start>", "me", "so", "cool", "<end>"]
l3 = ["thingo", "guy"]

s.update(l1)
s.update(l2)
# s.update([589,10,16,3])
print(s)
l1 = l3
s.update(l1)
l1 = l2
s.update(l2)
print(s)

{'cool', 'me', 'so', '<end>', '<start>', 'courez'}
{'cool', 'me', 'so', 'guy', '<end>', '<start>', 'thingo', 'courez'}


In [0]:
# preprocessing and stuff
PATH = "/gdrive/My Drive/data/fra.txt"
import spacy
import unicodedata
import itertools

from spacy.tokenizer import Tokenizer
EN = spacy.load('en')
FR = spacy.load('fr')

num_samples = 1000

en_tokenizer = Tokenizer(EN.vocab)
fr_tokenizer = Tokenizer(FR.vocab)

lines = ""
with open(PATH) as f:
  lines = f.read().split("\n")

w2i_inp = {}
i2w_inp = {}
w2i_out = {}
i2w_out = {}

punc = "?.!"
start_tag = "<start>"
end_tag = "<end>"
pad_tag = "<pad>"

max_inp_len = 0
max_out_len = 0
max_inp_phrase = ""
max_out_phrase = ""

all_inp_tokens = set()
all_out_tokens = set()
all_tar_tokens = set()

inp_seqs = []
out_seqs = []

num_inp_tokens = 0
num_out_tokens = 0

# Converts the unicode file to ascii
# from https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb#scrollTo=rd0jw-eC3jEh
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# what to do about start and end tags

for line in lines[:num_samples]:
  if len(line) > 1:
    inp_txt, out_txt = line.lower().split('\t')
    
    inp_txt = unicode_to_ascii(inp_txt)
    out_txt = unicode_to_ascii(out_txt)
    
    inp_txt = start_tag + " " + inp_txt.strip(punc).strip() + " " + end_tag
    out_txt = start_tag + " " + out_txt.strip(punc).strip() + " " + end_tag
    
    inp_tokens = list(en_tokenizer(inp_txt))
    out_tokens = list(fr_tokenizer(out_txt))
    
    stringed_inp = list(map(lambda t: str(t), inp_tokens))
    stringed_out = list(map(lambda t: str(t), out_tokens))
    
    if len(stringed_inp) > max_inp_len:
      max_inp_len = len(stringed_inp)
      max_inp_phrase = stringed_inp
      
    if len(stringed_out) > max_out_len:
      max_out_len = len(stringed_out)
      max_out_phrase = stringed_out
    
    inp_seqs.append(stringed_inp)
    out_seqs.append(stringed_out)
    
   
    
    # store the entire vocab
#     all_inp_tokens.update(*inp_tokens)
#     all_out_tokens.update(*out_tokens)
    all_inp_tokens.update(stringed_inp)
    all_out_tokens.update(stringed_out)
    

      
# print(all_out_tokens)
# print(all_inp_tokens)

# start at one because zero is padding
for i, (inp_t, out_t) in enumerate(itertools.zip_longest(all_inp_tokens, all_out_tokens), 1):
  
  if inp_t != None:
    i2w_inp[i] = inp_t
    w2i_inp[inp_t] =  i
    
  if out_t != None:
    i2w_out[i] = out_t
    w2i_out[out_t] =  i
  

# add padding token
i2w_inp[0] = pad_tag
w2i_inp[pad_tag] = 0
i2w_out[0] = pad_tag
w2i_out[pad_tag] = 0
    
num_inp_tokens = len(w2i_inp)    
num_out_tokens = len(w2i_out)    

# the time delay??
def process_seq(seqs, w2i, max_seq_len):
  # one hot encoding for now
  # processed = torch.zeros((num_samples, max_seq_len, len(w2i.items())))
  processed = torch.zeros((len(seqs), max_seq_len), dtype=torch.long)
  
  for i, s in enumerate(seqs[-1::-1]):
    for j, w in enumerate(s):
      processed[i, j] = w2i[w]
      
  return processed

# with size num_samples x max_seq_len x num_tokens
processed_inp = process_seq(inp_seqs, w2i_inp, max_inp_len)
processed_out = process_seq(out_seqs, w2i_out, max_out_len)

In [45]:
print(max_inp_len)
print(max_inp_phrase)
print(max_out_len)
print(max_out_phrase)

3
['<start>', 'go', '<end>']
4
['<start>', 'ca', 'alors', '<end>']


In [0]:

embedding_size = 200
hidden_size = 256
inp_vocab_size = len(w2i_inp)
out_vocab_size = len(w2i_out)
num_layers = 1 # tried 4 previously
num_directions = 2
bs = 2

# get all the shapes right
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.embeds = nn.Embedding(inp_vocab_size, embedding_size)
    self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=num_layers, bidirectional=True)
    
  def forward(self, inp):
    # max_inp_len, bs, embedding_size
    emb = self.embeds(inp)
    # out -> bs, max_inp_len (2 * hidden_size)
    # reference directions with out.view(seq_len, bs, num_directions, hidden_size) -> 0 or 1 for direction
    
    # hs -> num_layers * num_directions, bs, hidden_size
    # cs -> num_layers * num_directions, bs, hidden_size
    return self.lstm(emb)
  
# using Bahdanau attention
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embeds = nn.Embedding(out_vocab_size, embedding_size)
    self.tanh = nn.Tanh()
    self.v = nn.Linear(128, 1)
    self.w1 = nn.Linear(num_directions * hidden_size, 128)
    self.w2 = nn.Linear(hidden_size, 128)
    
    
    self.d = nn.Linear(hidden_size, out_vocab_size) # to get into the right shape
    self.lstm = nn.LSTM(num_directions * hidden_size + embedding_size, hidden_size, num_layers=num_layers)
    
    # what is hs versus ht
    # you probably have to feed this one at a time
  def forward(self, x, state, enc_outputs):
    # enc_outputs -> bs, max_inp_len, num_directions * hidden_size
    # x -> 1
    # h -> max_inp_len, hidden_size <- transform this? I want something 2d
    # w1 * enc_outputs -> bs, max_inp_len, 128
    # w2 * h -> max_inp_len, 128
    # N = (num_layers * num_directions)
    # cat_vec -> bs, max_inp_len 128
    # scores -> bs, max_inp_len  1
    # attn_weights -> bs, max_inp_len, 1
    hidden = state[0]
    h = torch.mean(hidden, 0)
    cat_vec = self.w1(enc_outputs) + self.w2(h).unsqueeze(0) #which dim?
    scores = self.v(self.tanh(cat_vec))
    attn_weights = F.log_softmax(scores, dim=1)
    
#     print("hidden: {}".format(hidden.shape))
#     print("enc out: {}".format(enc_outputs.shape))
#     print("attn weights: {}".format(attn_weights.shape))
    
    # bs, max_inp_len, num_directions * hidden_size
    context = attn_weights * enc_outputs
    context = torch.sum(context, 1).view(enc_outputs.shape[0], 1, -1)
    
#     print("context shape: {}".format(context.shape))
    
    # bs, 1, embedding_size
    x_emb = self.embeds(x).unsqueeze(1)
    
    
    # bs, 1, embedding_size + num_directions * hidden_size
    x_emb_and_context = torch.cat((x_emb, context), -1)
    
#     print(x_emb_and_context.shape)
    
    # bs, 1, num_directions * hidden_size
    out, (hs, cs) = self.lstm(x_emb_and_context)
    
    # bs, 1, out_vocab_size
    out = self.d(out)

    
    return F.log_softmax(out, dim=-1), (hs, cs)

In [157]:
print(max_inp_len)
print(max_out_len)

print(num_samples // 2)

5
9
500


In [159]:
enc = Encoder()
dec = Decoder()

loss_func = nn.CrossEntropyLoss()
accuracy = 0
optimizer = torch.optim.RMSprop([*enc.parameters(), *dec.parameters()], lr=0.0005)

# split the data into a validation set
# 20%? <- I'm not sure that works here, given, I don't think this will generalize...?
# basically, the system can't be expected to generate a translation for things it's never seen before...
def validation(epoch, teacher_forcing=True):
  # not sure yet
  return

def train(epoch, teacher_forcing=True):
  total_loss = 0
  
  for i in range((num_samples) // bs):
    
    batch_input = processed_inp[i*bs:i*bs + bs]
    batch_output = processed_out[i*bs:i*bs + bs]
    
    enc_outputs, (hs, cs) = enc(batch_input)
    
    dec_input = batch_output[:,0] #start tags
    
    loss = 0
    
    for t in range(1, max_out_len):
      dec_out, (hs, cs) = dec(dec_input, (hs, cs), enc_outputs)
      
      loss += loss_func(dec_out.squeeze(1), batch_output[:,t])

      if teacher_forcing:
        dec_input = batch_output[:,t] #feed targets
      else:
        dec_input = dec_out #feed predictions
      
      
    loss.backward()
    
    total_loss += loss.item()
    
    optimizer.step()
    optimizer.zero_grad()
    
  return total_loss
    
epochs = 200

for e in range(epochs):
  all_loss = train(e, teacher_forcing=True)
  
  if e % 10 == 0:
    print(all_loss)

7348.703876018524
523.3971020504832
196.82568502845243
157.39631367329275
140.0397917900409
134.64355736157813
126.34079016406417
125.65663608152897
134.36963966869484
123.47220421350539
124.0023149696338
131.8164099638384
133.40924191668947
125.56055317252326
140.52100048771132
116.55861834327779
116.85155664656153
118.9114346204124
115.85045656959122
117.14848609665398


In [165]:
# decode
# s = inp_seqs[6]
# print(s)

# eng seq
def process_seq(seq):
  uni_seq = unicode_to_ascii(seq)
  uni_seq = start_tag + " " + uni_seq.strip(punc).strip() + " " + end_tag
  processed = list(en_tokenizer(uni_seq))
  processed = map(lambda s: str(s), processed)
#   processed = s
  
  tensor = torch.zeros((1, max_inp_len), dtype=torch.long)
  
  for i, w in enumerate(processed):
    tensor[:, i] = w2i_inp[w]
    
#   print(tensor)
  
  translation = ""
  meta_trans = [start_tag]
  idx = 0
  enc_out, (hs, cs) = enc(tensor)
  dec_inp = torch.tensor([w2i_inp[start_tag]])
  
#   print(dec_inp.shape)
#   print(enc_out.shape)
  
  while idx < max_out_len:
    dec_out, (hs, cs) = dec(dec_inp, (hs, cs), enc_out)
    word_idx = torch.argmax(dec_out, dim=-1)
    word = i2w_out[int(word_idx[0][0])]
    meta_trans.append(word)
#     print(word_idx[0].shape)
    dec_inp = word_idx[0] 
    if word == end_tag or word == pad_tag:
#       print("produced end/pad tag")
      return translation.strip(), meta_trans # we're done!
    else:
      translation += (word + " ")
    idx += 1
  return translation.strip(), meta_trans
      
      
t, m = process_seq("")
print(t)
print(m)

pars d'ici
['<start>', 'pars', "d'ici", '<end>']
